# Chapter 3 - Basing Software Development on Reusable Technology
## Building on the Experience of Others
- Software engineers should avoid re-developing software already developed.

### Types of reuse:
- Reuse of **expertise**.
- Reus of **standard designs** and algorithms.
- Reuse of **libraries** of classes or procedures.
- Reuse of powerful **commands** built into languages and operating systems.
- Reuse of **frameworks**.
- Reuse of complete **applications**.

## Frameworks: Reusable Subsystems
- A *framework* is reusable software that implements a generic solution to a generalized problem.
- It provides common facilities applicable to different application programs.

**Principle**: Applications that do different, but related, things tend to have similar designs.

### Frameworks to promote reuse
- A framework is intrinsically *incomplete*.
- Certain classes or methods are used by the framework, but are missing (*slots*). It is mandatory to fill slots (abstract methods).
- Some functionality is optional.
    - Allowance is made for developer to provide it (*hooks* or *extension points*).
- Developers use the *services* that the framework provides.
    - Taken together the services are called the Application Program Interface (*API*).

### Object-oriented Frameworks
- In the object oriented paradigm, a framework is composed of a library of classes.
- The API is defined by the set of all **public methods** of these classes.
- Some of the classes will normally be abstract and there are often many Interfaces.

### Frameworks and Product Lines
- A product line (or product family) is a set of products built on a common base of technology.
- The various products in the product line have **different features** to satisfy different markets.
- The software **common to all products** is included in a framework.
- Each product is produced by **filling the available hooks and slots**.

### Types of Frameworks
- A *horizontal* framework provides general application facilities that a large number of applications can use.
- A *vertical* framework (*application framework*) is more 'complete' but still needs some slots to be filled to adapt it to specific application needs.

### The Client-Server Architecture
**A *distributed system* is a system in which**:
- Computations are performed by *separate programs*.
- Normally running on separate pieces of hardware that co-operate to perform the task of the system.

**Server**:
- A program that *provides a service* for other programs that connect to it using a communication channel.

**Client**:
- A program that accesses a server *to obtain services*.
- A server may be accessed by many clients simultaneously.

### Activities of a Server
1. Initializes itself.
2. Starts listening for clients.
3. Handles the following types of events originating from clients:
    - Accepts connections
    - Responds to messages
    - Handles client disconnection
4. May stop listening.
5. Must cleanly terminate.

### Activities of a Client
1. Initializes itself.
2. Initiates a connection.
3. Sends messages.
4. Handles the following types of events originating from the server:
    - Responds to messages
    - Handles server disconnection
5. Must cleanly terminate.

### Threads in a Client-Server System
**Client Side**
- Client will always need at least 2 threads.
- One for interacting with the user.
- One for waiting for server events.

**Server Side**
- Thread waiting for connections.
- Thread waiting for messages from a client.
- Thread interacting with server user.

### Thin- vs. Fat-Client Systems
**Thin-client system**
- Client is made as small as possible.
- Most of the work is done in the server.
- Client easy to download over the network.

**Fat-client system**
- As much work as possible is delegated to the clients.
- Server can handle more clients.



### Communication Protocols
- The messages the client sends to the server form a *language*.
    - The server has to be programmed to understand that language.
- The messages the server sends to the client also form a language.
    - The client has to be programmed to understand that language.
- When a client and server are communicating, they are in effect having a conversation using these two languages.
- The two languages and the rules of the conversation, taken together, are called the *protocol*.

### Technology Needed to Build Client-Server Systems
**Internet Protocol (IP)**
- Route messages from one computer to another.
- Long messages are normally split up into small pieces.

**Transmission Control Protocol (TCP)**
- Handles *connections* between two computers.
- Computers can then exchange many IP messages over a connection.
- Assures that the messages have been satisfactorily received.

**A host has an *IP address* and a *host name***
- Several servers can run on the same host.
- Each server is identified by a port numer (0 to 65535).
- To initiate communication with a server, a client must know both the host name and the port number.

### Establishing a connection in Java
- The `java.net` package permits the creation of a TCP/IP connection between two applications.

**Before a connection can be established, the server must start *listening* to one of the ports:**

```java
ServerSocket serverSocket = new ServerSocket(port);
Socket clientSocket = serverSocket.accept();
```

**For a client to connect to a server:**

```java
Socket clientSocket = new Socket(host, port);
```

### Exchanging information in Java
- Each program uses an instance of
    - `InputStream` to receive messages from the other program.
    - `OutputStream` to send messages to the other program.
    - These are found in package `java.io`.

```java
output = clientSocket.getOutputStream();
input = clientSocket.getInputStream();
```

### Sending and Receiving Messages
- Without any filters (raw bytes)

```java
output.write(msg);
msg = input.read();
```
or
```java
output.writeDouble(msg);
msg = input.readDouble();
```
or
```java
output.writeObject(msg);
msg = input.readObject();
```

### The Object Client-Server Framework (OCSF)

![OCSF Classes](../Resources/OCSF_Classes.png)

**Software engineers using OCSF *never* modify its three classes. They:**
- *Create subclasses* of the abstract classes in the framework.
- *Call public methods* that are provided by the framework.
- *Override* certain slot and hook methods (explicitly designed to be overridden).

### The Client Side
- **Consists of a single class:** `AbstractClient`
- *Must* be subclassed.
- Any subclass must provide an implementation for `handleMessageFromServer`.
- Implements the `Runnable` interface.
    - Has a `run` method which contains a loop that executes for the lifetime of the thread.

### Using `AbstractClient`
- Create a subclass of `AbstractClient`.
- Implement `handleMessageFromServer` slot method.
- Write code that:
    - Creates an instance of the new subclass.
    - Calls `openConnection`.
    - Sends messages to the server using the `sendToServer` service method.
- Implement the `connectionClosed` callback.
- Implement the `connectionException` callback.

### Internals of `AbstractClient`
- Instance variables:
    - A `Socket` which keeps all the information about the connection to the server.
    - Two streams, an `ObjectOutputStream` and an `ObjectInputStream`.
    - A `Thread` that runs using `AbstractClient`'s run method.
    - Two variables storing the *host* and *port* of the server.

### The Server Side
- Two classes:
    - One for the thread which listens for new connections (`AbstractServer`).
    - One for the threads that handle the connections to clients (`ConnectionToClient`).

### Using `AbstractServer` and `ConnectionToClient`
- Create a subclass of `AbstractServer`.
- Implement the slot method `handleMessageFromClient`.
- Write code that:
    - Creates an instance of the subclass of `AbstractServer`.
    - Calls the `listen` method.
    - Sends messages to clients using hte `getClientConnections` and `sendToClient` service methods, or `sendToAllClients`.
- Implement one or more callback methods.

### Internals of `AbstractServer` and `ConnectionToClient`
- The `setInfo` and `getInfo` methods make use of a Java class called a `HashMap`.
- Many methods in the server side are `synchronized`.
- The collection of instances of `ConnectionToClient` is stored using a special class called `ThreadGroup`.
- The server must pause from listening every 500ms to see if the `stopListening` method has been called. If not, then it resumes listening immediately.

### An Instant Messageing Application: SimpleChat

![SimpleChat UML Diagram](../Resources/SimpleChatUML.png)

- `EchoServer` is a subclass of `AbstractServer`.
- The `main` method creates a new instance and starts it. It listens for clients and handles connections until the server is stopped.
- The three *callback* methods just print out a message to the user.
- The *slot* method `handleMessageFromClient` calls `sendToAllClients`.

- When the client program starts, it creates instances of two classes: `ChatClient` and `ClientConsole`.
- `ChatClient`:
    - Is a subclass of `AbstractClient`.
    - Overrides `handleMessageFromServer`.
- `ClientConsole`:
    - Is the user interface class that implements the interface `ChatIF`.
    - Accepts user input by calling `accept` in its `run` method.
    - Sends all user input to the `ChatClient` by calling its `handleMessageFromClientUI`.